<a href="https://colab.research.google.com/github/pratik-poudel/datacrunch/blob/main/fork_of_machinehack_wipro_combinations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pip install feature-engine

In [1]:
pip install optuna

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold, TimeSeriesSplit, GroupKFold, StratifiedKFold
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor, RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import lightgbm as lgb
from tqdm import tqdm

from scipy import stats
import warnings
warnings.filterwarnings('ignore')

from itertools import combinations, permutations, combinations_with_replacement

import optuna
from IPython.display import clear_output

pd.set_option('display.max_columns', None)

/usr/local/lib/python3.7/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:17: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  "Since version 1.0, "


# Group Time Series

In [3]:
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class GroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_size : int, default=None
        Maximum size for a single training set.
    Examples
    --------
    >>> import numpy as np
    >>> from sklearn.model_selection import GroupTimeSeriesSplit
    >>> groups = np.array(['a', 'a', 'a', 'a', 'a', 'a',\
                           'b', 'b', 'b', 'b', 'b',\
                           'c', 'c', 'c', 'c',\
                           'd', 'd', 'd'])
    >>> gtss = GroupTimeSeriesSplit(n_splits=3)
    >>> for train_idx, test_idx in gtss.split(groups, groups=groups):
    ...     print("TRAIN:", train_idx, "TEST:", test_idx)
    ...     print("TRAIN GROUP:", groups[train_idx],\
                  "TEST GROUP:", groups[test_idx])
    TRAIN: [0, 1, 2, 3, 4, 5] TEST: [6, 7, 8, 9, 10]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a']\
    TEST GROUP: ['b' 'b' 'b' 'b' 'b']
    TRAIN: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10] TEST: [11, 12, 13, 14]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a' 'b' 'b' 'b' 'b' 'b']\
    TEST GROUP: ['c' 'c' 'c' 'c']
    TRAIN: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]\
    TEST: [15, 16, 17]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a' 'b' 'b' 'b' 'b' 'b' 'c' 'c' 'c' 'c']\
    TEST GROUP: ['d' 'd' 'd']
    """
    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_size=None
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_size = max_train_size

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))
        group_test_size = n_groups // n_folds
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []
            for train_group_idx in unique_groups[:group_test_start]:
                train_array_tmp = group_dict[train_group_idx]
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)
            train_end = train_array.size
            if self.max_train_size and self.max_train_size < train_end:
                train_array = train_array[train_end -
                                          self.max_train_size:train_end]
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)
            yield [int(i) for i in train_array], [int(i) for i in test_array]

# Load Data

In [5]:
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')

In [6]:
cols = ['Year', 'Month', 'Day', 'Hour', 'Minute', 'Cloud Type',
        'Dew Point', 'Temperature', 'Pressure',
        'Relative Humidity', 'Solar Zenith Angle','Precipitable Water', 
        'Wind Direction', 'Wind Speed', 
        'Clearsky DHI', 'Clearsky DNI', 'Clearsky GHI']
train = train[cols]
test = test[cols]

In [7]:
col_names = ["_".join(i) for i in train.columns.str.lower().str.split(" ")]
train.columns = col_names
test.columns = col_names

In [8]:
# pd.pivot_table(train,index='month', columns='year', values='clearsky_dhi').plot(subplots=True, figsize=(15, 10))
# plt.show()

In [9]:
# pd.pivot_table(train,index='month',columns='year', values='clearsky_dni').plot(subplots=True, figsize=(15,10))
# plt.show()

In [10]:
# pd.pivot_table(train,index='month', columns='year', values='clearsky_ghi').plot(subplots=True, figsize=(15, 10))
# plt.show()

# Clean Data Sets

In [11]:
def find_boundaries(df_train,df_test, variable, distance):
    
    IQR = df_train[variable].quantile(0.75) - df_train[variable].quantile(0.25)
    lower_boundary = df_train[variable].quantile(0.25) - (IQR * distance)
    upper_boundary = df_train[variable].quantile(0.75) + (IQR * distance)
    print(lower_boundary, upper_boundary)
    #cap at min and maximum values
    df_train[variable] = np.where(df_train[variable] > upper_boundary,upper_boundary,
                                     np.where(df_train[variable] < lower_boundary,lower_boundary, df_train[variable]))
    df_test[variable] = np.where(df_test[variable] > upper_boundary,upper_boundary,
                                     np.where(df_test[variable] < lower_boundary,lower_boundary, df_test[variable]))
    
#     display(df.head())
    return df_train, df_test

In [12]:
train, test = find_boundaries(train,test, 'dew_point', 1)
train, test = find_boundaries(train, test,'temperature', 1.5)
train, test = find_boundaries(train, test,'relative_humidity', 1.5)
train, test = find_boundaries(train, test,'wind_speed', 1.5)
train, test = find_boundaries(train, test,'pressure', 0.75)

-5.0 34.0
-7.5 44.5
17.050000000000004 146.49
-1.2499999999999998 5.55
992.5 1017.5


In [13]:
def get_box_plots(df, columns, name='Train'):
    plt.figure(figsize=(20, 14))
    plt.subplots_adjust(hspace=0.4)
    plt.suptitle(f"{name} Box Plots Of Float Columns", fontsize=18, y=0.95)

    # loop through the length of tickers and keep track of index
    for n, ticker in enumerate(columns):

        ax = plt.subplot(3, 4, n + 1)
        df[ticker].plot(kind='box', ax=ax, )
    plt.show()

In [14]:
# get_box_plots(test, train.select_dtypes('float').columns)

In [15]:
# get_box_plots(test, tr.select_dtypes('float').columns)

In [16]:
def get_hist_plots(df, columns, name="Train" ):
    plt.figure(figsize=(20,14))
    plt.subplots_adjust(hspace=0.4)
    plt.suptitle(f"{name}", fontsize=18, y=0.96)
    for ind, col in enumerate(columns):
        ax = plt.subplot(3, 4 , ind+1)
        train[col].plot(kind='hist')
        ax.set_title(col)

In [17]:
# get_hist_plots(train, train.select_dtypes('float').columns)

# get_hist_plots(test, train.select_dtypes('float').columns, name= "Test")

In [18]:
def get_rare_categories(df, column, percentage = 0.05):
    plt.figure(figsize=(12, 6))
    fig = train[column].value_counts(normalize=True).plot.bar()
    fig.axhline(y=0.08, color='red')
    fig.set_title(f'Identifying Rare Categories For {column}')
    plt.show()

In [19]:
def find_frequent_labels(df, variable, tolerance):
    temp = df[variable].value_counts() / len(df)
    frequent = [x for x in temp.loc[temp>tolerance].index.values]
    return frequent

In [20]:
# # Infrequent Categories
# frequent_cat = find_frequent_labels(train, 'fill_flag', 0.05)
# frequent_cat

In [21]:
train['cloud_type'] = train['cloud_type'].replace(to_replace=[1, 7, 8, 9, 10], value=[2,7,7,7,7 ])
test['cloud_type'] = test['cloud_type'].replace(to_replace=[8, 9], value=[7,7 ])

In [22]:
train['season'] = train['month'].replace(to_replace=[3, 4, 5, 
                                   6, 7, 8, 
                                   9, 10, 11, 
                                   12, 1, 2], 
                       value=[1, 1,1,
                              2, 2, 2, 
                              3, 3, 3, 
                              4, 4, 4])

train['daytime'] = train['hour'].apply(lambda x: 1 if x >=6 and x <=18 else 0)

test['season'] = test['month'].replace(to_replace=[3, 4, 5, 
                                   6, 7, 8, 
                                   9, 10, 11, 
                                   12, 1, 2], 
                       value=[1, 1,1,
                              2, 2, 2, 
                              3, 3, 3, 
                              4, 4, 4])

test['daytime'] = test['hour'].apply(lambda x: 1 if x >=6 and x <=18 else 0)

In [23]:
features = ['year', 'season','month', 'day', 'hour', 'daytime','minute', 'cloud_type','dew_point',
       'temperature', 'pressure', 'relative_humidity', 'solar_zenith_angle',
       'precipitable_water', 'wind_direction', 'wind_speed']
y1 = train['clearsky_dhi']
y2 = train['clearsky_dni']
y3 = train['clearsky_ghi']

In [24]:
# train = train[features]
# test = test[features]

# More Features

In [25]:
def get_aggregations(df, groupby, name):
    feature_names = [ 'max', 'min','mean', 'median', 'std', 'mad']
    operations = ['max', 'min', 'mean', 'median', 'std', 'mad']
    all_df=[]
    
    for i in ['dew_point','temperature', 'pressure', 'relative_humidity', 
              'solar_zenith_angle','precipitable_water', 'wind_direction', 'wind_speed']:
#         print(i)
        temp = pd.DataFrame()
        temp[feature_names] =df.groupby(groupby)[i].agg(operations)
        temp.columns = [f'{name}_{i}_{j}' for j in operations]
        all_df.append(temp)
    all_df = pd.concat(all_df, axis=1).reset_index()
    df_ = pd.merge(df, all_df, left_on=groupby, right_on=groupby, how='left')
    return df_

In [26]:
train_= get_aggregations(train, ['year', 'month'],'month' )
train_ = get_aggregations(train_, ['year', 'month', 'day'],'day' )
train_ = get_aggregations(train_, ['year','season'],'season' )
train_ = get_aggregations(train_, ['year','month', 'day', 'daytime'],'daytime' )

In [27]:
test_= get_aggregations(test, ['year', 'month'],'month' )
test_ = get_aggregations(test_, ['year', 'month', 'day'],'day' )
test_ = get_aggregations(test_, ['year','season'],'season' )
test_ = get_aggregations(test_, ['year','month', 'day', 'daytime'],'daytime' )

In [28]:
original = ['year', 'season', 'month', 'day', 'hour', 'daytime', 'minute',
       'cloud_type', 'dew_point', 'temperature', 'pressure',
       'relative_humidity', 'solar_zenith_angle', 'precipitable_water',
       'wind_direction', 'wind_speed']

In [30]:
corr_features = [col for col in train_.columns if col not in original + ['clearsky_dhi', 'clearsky_dni', 'clearsky_ghi']]

In [31]:
corr_matrix = train_[corr_features].corr().abs()

upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape),k=1).astype(np.bool))

to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]

print(to_drop)

['month_dew_point_median', 'month_dew_point_std', 'month_dew_point_mad', 'month_temperature_min', 'month_temperature_mean', 'month_temperature_median', 'month_temperature_mad', 'month_pressure_mad', 'month_relative_humidity_median', 'month_relative_humidity_mad', 'month_solar_zenith_angle_min', 'month_solar_zenith_angle_mean', 'month_solar_zenith_angle_median', 'month_solar_zenith_angle_mad', 'month_precipitable_water_mean', 'month_precipitable_water_median', 'month_precipitable_water_mad', 'month_wind_direction_median', 'month_wind_direction_mad', 'month_wind_speed_median', 'month_wind_speed_mad', 'day_dew_point_mean', 'day_dew_point_median', 'day_dew_point_mad', 'day_temperature_min', 'day_temperature_mean', 'day_temperature_median', 'day_temperature_mad', 'day_pressure_median', 'day_pressure_mad', 'day_relative_humidity_median', 'day_relative_humidity_mad', 'day_solar_zenith_angle_max', 'day_solar_zenith_angle_min', 'day_solar_zenith_angle_mean', 'day_solar_zenith_angle_median', 'da

In [32]:
train_ = train_.drop(columns=to_drop, axis=1)
test_ = test_.drop(columns=to_drop, axis=1)

In [34]:
train_.shape, test_.shape

((175296, 114), (17520, 114))

In [77]:
# train_df = pd.concat([train_, y1, y2, y3], axis=1)

# Encode Categorical Data

In [35]:
!pip install category_encoders

In [36]:
import category_encoders

In [39]:
train_df = train_.copy()
list(range(2009, 2019))

[2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]

In [40]:
train_data = train_df[train_df.year.isin([2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016])]
val_data = train_df[train_df.year.isin([2017, 2018])]

In [41]:
cat = ['season','month','day','hour','daytime','minute','cloud_type']
encoded_df_train = train_data[cat]
encoded_df_val = val_data[cat]
encoded_df_test = test_[cat]

In [42]:
encoder = category_encoders.CatBoostEncoder(cols=cat)
encoder.fit(encoded_df_train, train_data['clearsky_dni'])

CatBoostEncoder(cols=['season', 'month', 'day', 'hour', 'daytime', 'minute',
                      'cloud_type'])

In [43]:
features_train = encoder.transform(encoded_df_train)
features_val = encoder.transform(encoded_df_val)
features_test = encoder.transform(encoded_df_test)

In [44]:
fdf = pd.concat([features_train, features_val])

In [45]:
train_df = train_df.drop(columns=cat)

In [46]:
train_df = pd.concat([fdf, train_df], axis=1)

In [47]:
test_= test_.drop(columns= cat)

In [48]:
test_df = pd.concat([features_test, test_], axis=1)

# Combinations

In [164]:
def run_lgb(X , y, test_data ):
    
    params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mean_squared_error',
    'random_state':2021,
    'verbose':-1, 
    'early_stopping_rounds':50,
    'num_leaves': 40, 
    'max_depth': 4, 
    # 'min_child_weight': 5, 
    # 'feature_fraction': 0.7398835483954241, 
    # 'bagging_fraction': 0.5287430772867108,
    # 'bagging_freq': 6, 
    # 'min_child_samples': 51, 
    # 'lambda_l1': 1.7798097405444633e-05,
    # 'lambda_l2': 1.3516742170658552e-05, 
    # 'learning_rate': 0.09771288727057821, 
    # 'max_bin': 114, 
    'num_boost_round': 300

 }
    ts =   KFold(n_splits=5, shuffle=True)

    oof_pred = np.zeros(len(X))
    test_pred = np.zeros(len(test_data))

    for tr_ind, val_ind in ts.split(X, y):
        X_train, X_val = X.iloc[tr_ind], X.iloc[val_ind]
        y_train, y_val = y.iloc[tr_ind], y.iloc[val_ind]


        tr_data = lgb.Dataset(X_train, y_train)
        va_data = lgb.Dataset(X_val, y_val)
        model = lgb.train(params = params,
                      train_set = tr_data, 
                      valid_sets = [tr_data, va_data],
                      verbose_eval = 300,
                     )

        val_pred = model.predict(X_val)
        oof_pred[val_ind] = val_pred
        test_pred += model.predict(test_data)
        
    mse= mean_squared_error(y, oof_pred)
    test_pred = test_pred / ts.n_splits
    return mse, oof_pred, test_pred

In [165]:
def run_combinations(cols_for_combination,length, target, train, test):
    comb = []
    oof_preds= []
    test_preds = []
    for i in combinations(cols_for_combination, length):
        comb.append(list(i))
    allmse = []
    names = names= ["_".join(i) for i in comb]
    for cols in tqdm(comb):
        mse, oof, tpred = run_lgb(train[cols], train[target], test[cols])
        allmse.append(mse)
        oof_preds.append(oof)
        test_preds.append(tpred)
    df = pd.DataFrame(allmse).T
    df.columns = names
    oof_preds= np.column_stack(oof_preds)
    test_preds= np.column_stack(test_preds)
    oof_preds = pd.DataFrame(oof_preds, columns=df.columns)
    test_preds = pd.DataFrame(test_preds, columns=df.columns)
    
    return df, oof_preds, test_preds

In [166]:
train_df.shape

(175296, 114)

In [181]:
columns = train_df.corrwith(train_df['clearsky_dni']).abs().sort_values().tail(20).index.to_list()

In [182]:
# columns

In [183]:
columns.remove('clearsky_dni')
columns.remove('clearsky_dhi')
columns.remove('clearsky_ghi')

In [184]:
# columns= ['hour', 'solar_zenith_angle','relative_humidity']

In [ ]:
three_comb_score, oof_pred_df3, test_pred_df3 = run_combinations(columns, 3, 'clearsky_dni', train_df, test_df)

In [ ]:
mse_dhi = run_model(model_params=p, y=y1)
mse_dni = run_model(model_params=p, y=y2)
mse_ghi = run_model(model_params=p, y=y3)
print(mse_dhi, mse_dni, mse_ghi)
print((mse_dhi + mse_dni + mse_ghi)/ 3)

In [ ]:
def collect_predictions(model_params, y, test_data):
    y_true =[]
    predicted_y = []
    features = ['month', 'day', 'hour', 'minute', 'cloud_type', 'dew_point',
       'temperature', 'pressure', 'relative_humidity', 'solar_zenith_angle',
       'precipitable_water', 'wind_direction', 'wind_speed', 'fill_flag', 'year']
    test_data = test_data[features]
    test_data = test_data.drop(columns=['year'])
    test_predictions = np.zeros((len(test_data)))
    for tr_ind, val_ind in gts.split(X, y, groups=X['year']):
        X_train, X_val = X.iloc[tr_ind], X.iloc[val_ind]
        y_train, y_val = y.iloc[tr_ind], y.iloc[val_ind]
        X_train = X_train.drop(columns=['year'])
        X_val= X_val.drop(columns=['year'])

        tr_data = lgb.Dataset(X_train, y_train)
        va_data = lgb.Dataset(X_val, y_val)
        
        model_dni = lgb.train(params = model_params,
                          train_set = tr_data, 
                          valid_sets = [tr_data, va_data],
                          verbose_eval = None,
                             )
        val_pred_dni = model_dni.predict(X_val,num_iteration=model_dni.best_iteration)
        y_true.append(y_val.values)
        predicted_y.append(val_pred_dni)
        test_predictions += model_dni.predict(test_data,num_iteration=model_dni.best_iteration)
    test_predictions = test_predictions/ gts.n_splits
    y_true = np.hstack(y_true)
    predicted_y = np.hstack(predicted_y)
    mse_dni= mean_squared_error(y_true, predicted_y)
    
    return mse_dni, test_predictions

In [ ]:
mse_dhi, test_pred_dhi, = collect_predictions(model_params=p, y=y1, test_data = test)
mse_dni, test_pred_dni= collect_predictions(model_params=p, y=y2, test_data = test)
mse_ghi, test_pred_ghi = collect_predictions(model_params=p, y=y3, test_data = test)

In [ ]:
mse_dhi, mse_dni, mse_ghi

In [ ]:
(mse_dhi+ mse_dni+ mse_ghi)/ 3

In [ ]:
sub = pd.DataFrame()
sub['Clearsky DHI'] = test_pred_dhi
sub['Clearsky DNI'] = test_pred_dni
sub['Clearsky GHI'] = test_pred_ghi

In [ ]:
sub.hist(figsize=(20, 6))
plt.show()
pd.concat([y1, y2, y3], axis=1).hist(figsize=(20, 6))
plt.show()

In [ ]:
sub.to_csv('my_submission_file.csv', index=False)

# RUN OPTUNA GROUP TIME SPLIT

In [ ]:
gts = GroupTimeSeriesSplit(n_splits=5)

def run_model(model_params,X, y):
    y_true =[]
    predicted_y = []

    for tr_ind, val_ind in gts.split(X, y, groups = X['year']):
        X_train, X_val = X.iloc[tr_ind], X.iloc[val_ind]
        y_train, y_val = y.iloc[tr_ind], y.iloc[val_ind]

        X_train = X_train.drop(columns=['year'])
        X_val= X_val.drop(columns=['year'])
                
        cat = ['season','month','day','hour','daytime','minute','cloud_type']
        encoded_df_train = X_train[cat]
        encoded_df_val = X_val[cat]
        encoder = category_encoders.CatBoostEncoder(cols=cat)
        encoder.fit(encoded_df_train, y_train)
        features_train = encoder.transform(encoded_df_train).reset_index(drop=True)
        features_val = encoder.transform(encoded_df_val).reset_index(drop=True)
        
        not_to_scale = ['year','season','month','day','hour','daytime','minute','cloud_type',]
        to_scale = [col for col in X.columns if col not in not_to_scale ]
        ss = StandardScaler()
        train_scaled = pd.DataFrame(ss.fit_transform(X_train[to_scale]), columns=to_scale).reset_index(drop=True)
        val_scaled = pd.DataFrame(ss.transform(X_val[to_scale]), columns=to_scale).reset_index(drop=True)
        

        X_train = pd.concat([features_train,train_scaled ], axis=1)
        X_val = pd.concat([features_val,val_scaled], axis=1)

        tr_data = lgb.Dataset(X_train, y_train)
        va_data = lgb.Dataset(X_val, y_val)
        
        model_dni = lgb.train(params = model_params,
                          train_set = tr_data, 
                          valid_sets = [tr_data, va_data],
                          verbose_eval = None,
                             )
        val_pred_dni = model_dni.predict(X_val,num_iteration=model_dni.best_iteration)
        y_true.append(y_val.values)
        print(mean_squared_error(np.expm1(y_val), np.expm1(val_pred_dni)))
        predicted_y.append(val_pred_dni)
    y_true= np.hstack(y_true)
    predicted_y = np.hstack(predicted_y)    
    mse = mean_squared_error(np.expm1(y_true), np.expm1(predicted_y))
    return mse


In [ ]:
def objective(trial):    
    params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mean_squared_error',
    'early_stopping_rounds': 10, 
    'random_state': 2021, 
    'verbose':-1,
    'num_leaves': trial.suggest_int('num_leaves', 4, 256),
    'max_depth': trial.suggest_int('max_depth', 4, 16),
    'min_child_weight': trial.suggest_int('min_child_weight', 1, 16),
    'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
    'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
    'bagging_freq': trial.suggest_int('bagging_freq', 1, 8),
    'min_child_samples': trial.suggest_int('min_child_samples', 4, 80),
    'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 1.0),
    'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 1.0),
    'learning_rate' : trial.suggest_uniform('learning_rate', 0.01, 0.99),
    'max_bin': trial.suggest_int('max_bin', 10, 200),
    'num_boost_round' :trial.suggest_int('num_boost_round', 500, 1500),        
#     'categorical_column': trial.suggest_categorical('categorical_column', [ 0, 1, 2, 3, 4, 13, 
#         [0, 1], [0, 2], [0, 3], [0, 4], [0, 13], [1, 2], [1, 3], [1, 4], [1, 13], [2, 3], [2, 4], [2, 13], [3, 4], [3, 13], [4, 13], 
#         [0, 1, 2], [0, 1, 3], [0, 1, 4], [0, 1, 13], [0, 2, 3], [0, 2, 4], [0, 2, 13], [0, 3, 4], [0, 3, 13], [0, 4, 13], [1, 2, 3], [1, 2, 4], 
#         [1, 2, 13], [1, 3, 4], [1, 3, 13], [1, 4, 13], [2, 3, 4], [2, 3, 13], [2, 4, 13], [3, 4, 13], [0, 1, 2, 3], [0, 1, 2, 4], [0, 1, 2, 13],
#         [0, 1, 3, 4], [0, 1, 3, 13], [0, 1, 4, 13], [0, 2, 3, 4], [0, 2, 3, 13], [0, 2, 4, 13], [0, 3, 4, 13], [1, 2, 3, 4], [1, 2, 3, 13],
#         [1, 2, 4, 13], [1, 3, 4, 13], [2, 3, 4, 13], [0, 1, 2, 3], [0, 1, 2, 4], [0, 1, 2, 13], [0, 1, 3, 4], [0, 1, 3, 13], [0, 1, 4, 13], 
#         [0, 2, 3, 4], [0, 2, 3, 13], [0, 2, 4, 13], [0, 3, 4, 13], [1, 2, 3, 4], [1, 2, 3, 13], [1, 2, 4, 13], [1, 3, 4, 13], [2, 3, 4, 13]
#     ])
    }
    
    gts = GroupTimeSeriesSplit(n_splits=5)
    mse_dni = run_model(model_params=params,X=train_, y=y2_log)
    clear_output()
    return mse_dni

In [ ]:
%%time
N_TRIALS = 100

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=N_TRIALS, show_progress_bar= True)
study.best_params, study.best_value

In [ ]:
kf = StratifiedKFold(n_splits=5, shuffle=False, random_state=2319)
models = []
feature_importance_df = pd.DataFrame()
param =  {'num_leaves': 200,
             'objective': 'regression',
             'learning_rate': 0.03,
             'boosting': 'gbdt',
             'subsample': 0.5,
             'feature_fraction': 0.7,
             'n_jobs': -1,
             'seed': 50,
             'metric': 'rmse', 
          'verbose': -1
              }
p= {'boosting_type': 'gbdt',
 'objective': 'regression',
 'metric': 'mean_squared_error',
 'early_stopping_rounds': 10,
 'random_state': 2021,
 'verbose': -1}

oof = np.zeros(len(train_df))
  
for tr_idx, val_idx in tqdm(kf.split(X,X['year']), total = 5):
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    vl_x, vl_y = X.iloc[val_idx], y.iloc[val_idx]
    break
    tr_x = tr_x.drop(columns=['year'])
    vl_x = vl_x.drop(columns=['year'])
    
    tr_data = lgb.Dataset(tr_x, label = tr_y)
    vl_data = lgb.Dataset(vl_x, label = vl_y)
    clf = lgb.train(param, tr_data, 200, valid_sets = [tr_data, vl_data], verbose_eval = 100, 
                    early_stopping_rounds = 50)

    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = tr_x.columns
    fold_importance_df["importance"] = clf.feature_importance()

    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    models.append(clf)
    oof[val_idx] = clf.predict(vl_x)
mean_squared_error(pd.Series(np.expm1(oof)), y2)

In [ ]:
tr_x.year.unique()

In [ ]:
tr_x[tr_x['year'] == 2009]

In [ ]:
vl_x[vl_x.year == 2009]

In [ ]:
cols = (feature_importance_df[["feature", "importance"]]
        .groupby("feature")
        .mean()
        .sort_values(by="importance", ascending=False)[:20].index)
best_features = feature_importance_df.loc[feature_importance_df.feature.isin(cols)]

In [ ]:
plt.figure(figsize=(14,10))
sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance",ascending=False))
plt.title('LightGBM Features (averaged over folds)')
plt.tight_layout()

In [ ]:
y_true =[]
predicted_y = []
for tr_ind, val_ind in gts.split(X, y, groups=X['year']
                                              ):
    X_train, X_val = X.iloc[tr_ind], X.iloc[val_ind]
    y_train, y_val = y.iloc[tr_ind], y.iloc[val_ind]
    X_train = X_train.drop(columns=['year'])
    X_val= X_val.drop(columns=['year'])
    display(X_train.shape)

#     model = HistGradientBoostingRegressor()
#     model.fit(X_train, y_train)
    
    tr_data = lgb.Dataset(X_train, y_train)
    va_data = lgb.Dataset(X_val, y_val)

    model = lgb.train(params = p,
                      train_set = tr_data, 
                      valid_sets = [tr_data, va_data],
                      verbose_eval = 100,
                         )
    
    val_pred= model.predict(X_val)
#     print(mean_squared_error(y_val, val_pred))
    y_true.append(y_val.values)
    predicted_y.append(val_pred) 
mse= mean_squared_error(np.hstack(y_true), np.hstack(predicted_y))
mse

In [ ]:
pd.Series(model.feature_importance(), index=X_train.columns).sort_values(ascending=False)[:30]